# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
import time
from scipy.stats import linregress
from matplotlib import pyplot as plt

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("../output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,waitangi,-43.9535,-176.5597,12.12,97,100,0.45,NZ,1699372317
1,1,palikir - national government center,6.9248,158.1611,28.16,77,96,8.54,FM,1699372345
2,2,rio brilhante,-21.8019,-54.5464,38.05,16,2,3.47,BR,1699372129
3,3,invercargill,-46.4000,168.3500,9.45,85,100,0.45,NZ,1699372316
4,4,gaspe,48.8334,-64.4819,3.95,81,100,5.66,CA,1699372484


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [4]:
%%capture --no-display

# Configure the map plot
map_plot_1 = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 900,
    frame_height = 700,
)


# Display the map
map_plot_1

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [5]:
# Narrow down cities that fit criteria and drop any results with null values
narrowed_data = city_data_df[
    (city_data_df["Max Temp"] < 27) & (city_data_df["Max Temp"] > 22) \
    & (city_data_df["Wind Speed"] < 3) & (city_data_df["Cloudiness"] == 0)
]
# Drop any rows with null values
narrowed_data = narrowed_data.dropna()

# Display sample data
narrowed_data

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
16,16,shadipur julana,29.1333,76.4167,24.07,26,0,1.49,IN,1699372542
40,40,savannah,32.0835,-81.0998,24.42,48,0,2.06,US,1699372321
216,216,moires,35.0532,24.8744,22.84,72,0,1.34,GR,1699372539
229,229,crane,31.3974,-102.3501,24.13,30,0,2.66,US,1699372367
244,244,san elizario,31.5851,-106.2728,25.30,28,0,1.86,US,1699372546
282,282,sabalgarh,26.2500,77.4000,24.55,35,0,1.37,IN,1699372387
287,287,siwa oasis,29.2041,25.5195,26.78,36,0,1.96,EG,1699372408
301,301,mulayjah,27.2710,48.4242,25.68,44,0,2.71,SA,1699372607
341,341,salumbar,24.1333,74.0500,25.74,32,0,1.09,IN,1699372614
487,487,toliara,-23.3500,43.6667,25.55,75,0,1.94,MG,1699372647


### Step 3: Create a new DataFrame called `hotel_df`.

In [6]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = narrowed_data[["City", "Country", "Lat", "Lng", "Humidity"]].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df["Hotel Name"] = ""

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
16,shadipur julana,IN,29.1333,76.4167,26,
40,savannah,US,32.0835,-81.0998,48,
216,moires,GR,35.0532,24.8744,72,
229,crane,US,31.3974,-102.3501,30,
244,san elizario,US,31.5851,-106.2728,28,
282,sabalgarh,IN,26.2500,77.4000,35,
287,siwa oasis,EG,29.2041,25.5195,36,
301,mulayjah,SA,27.2710,48.4242,44,
341,salumbar,IN,24.1333,74.0500,32,
487,toliara,MG,-23.3500,43.6667,75,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
limit = 20
params = {
   "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit": limit
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make and API request using the params dictionaty
    name_address = requests.get(base_url, params=params)
    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
shadipur julana - nearest hotel: No hotel found
savannah - nearest hotel: Comfort Inn
moires - nearest hotel: Hotel Olympic
crane - nearest hotel: No hotel found
san elizario - nearest hotel: No hotel found
sabalgarh - nearest hotel: No hotel found
siwa oasis - nearest hotel: فندق الكيلانى
mulayjah - nearest hotel: No hotel found
salumbar - nearest hotel: Swagat Guest House
toliara - nearest hotel: Ambary
katsuren-haebaru - nearest hotel: No hotel found


,City,Country,Lat,Lng,Humidity,Hotel Name
16,shadipur julana,IN,29.1333,76.4167,26,No hotel found
40,savannah,US,32.0835,-81.0998,48,Comfort Inn
216,moires,GR,35.0532,24.8744,72,Hotel Olympic
229,crane,US,31.3974,-102.3501,30,No hotel found
244,san elizario,US,31.5851,-106.2728,28,No hotel found
282,sabalgarh,IN,26.2500,77.4000,35,No hotel found
287,siwa oasis,EG,29.2041,25.5195,36,فندق الكيلانى
301,mulayjah,SA,27.2710,48.4242,44,No hotel found
341,salumbar,IN,24.1333,74.0500,32,Swagat Guest House
487,toliara,MG,-23.3500,43.6667,75,Ambary


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [8]:
%%capture --no-display

# Configure the map plot
hotel_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "EsriWorldStreetMap",
    frame_width = 600,
    frame_height = 400,
    size = "Humidity",
    scale = 2,
    color = "City",
    hover_cols = ["Hotel Name", "Country"]
)

# Display the map
hotel_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)